In [1]:
from pymatgen.io.lammps.data import *

In [2]:
system = LammpsData.from_file("data.oleic_xligand.test")
xlig_dat = LammpsData.from_file("data.xligand")
oleic_dat = LammpsData.from_file("data.oleic_acid")

xlig = xlig_dat.structure
oleic = oleic_dat.structure

box, ff, tops = system.disassemble()
forcefield = ff.as_dict()

In [3]:
def get_ff_type(idx,structure):
    site = structure.as_dict()['sites']['sites'][idx] # preserves original index of atoms from lammps data file
    ff_type = site['properties']['ff_map']  # pull the forcefield type from the structure-topo dict
    return ff_type

def get_bond_type(bond,ff_dict):
    
    bond_type = ""
    
    for i in range(len(ff_dict['topo_coeffs']["Bond Coeffs"])):
        # print(bond, ff_dict['topo_coeffs']["Bond Coeffs"][i]['types'][0]) 
        if bond == ff_dict['topo_coeffs']["Bond Coeffs"][i]['types'][0]:
            bond_type = i
        elif bond[::-1] == ff_dict['topo_coeffs']["Bond Coeffs"][i]['types'][0]:
            bond_type = i
    
    if type(bond_type) == int:
        return bond_type
    else:
        raise Exception("type not found")

In [4]:
# this function expects:
# -- relation should be an array of atom types as imported by pymatgen from lammps datafile 
# -- eg [C1, H1, C1, N1]
# -- ff_dict should be the output of the ff.as_dict() pymatgen lammps forcefield object 
# -- topology should be the name of the relation you want to type (Bonds, Angles, Dihedrals, Impropers)
def get_topo_type(relation, ff_dict, topology):
    
    topo_type = ""
    topo_coeff = ""
    
    # convert standard topology name to singular + " Coeff" format for pymatgen ff_dict format
    topo = {"Bond", "Angle", "Dihedral", "Improper"}
    for i in topo:
        if i in topology:
            topo_coeff = i + " Coeffs"
    
    if topo_coeff == "":
        raise Exception("Topology not found")
    
    # check the original or reverse order of the topological relation (bond, angle...) 
    for i in range(len(ff_dict['topo_coeffs'][topo_coeff])):
        # print(relation, ff_dict['topo_coeffs'][topo_coeff][i]['types'][0])
        if relation in ff_dict['topo_coeffs'][topo_coeff][i]['types']:
            topo_type = i
        elif relation[::-1] in ff_dict['topo_coeffs'][topo_coeff][i]['types']:
            topo_type = i
    
    if type(topo_type) == int:
        return topo_type
    else:
        raise Exception("type not found")

In [5]:
dihed_typed = [get_ff_type(x,tops[0]) for x in tops[0].topologies["Dihedrals"][0]]
dihed_typed
get_topo_type(dihed_typed, forcefield, "Dihedrals")

3

In [6]:
# writes out the desired lammps topology in molecule-file compatible format 
# note the "+ 1"s to convert to lammps 1-based indexing
def get_lammps_topology(structure, ff_dict, topology):
    i = 0
    dat = []
    for neighbor in structure.topologies[topology]:
        i  += 1
        neighbor_types = [get_ff_type(x,structure) for x in neighbor] # get ff types for each element in the connected relation
        relation_type = get_topo_type(neighbor_types, ff_dict, topology) # get type of the relation
        lammps_bond = [str(i + 1) for i in neighbor] # convert to LAMMPS 1-based indexing
        # print(i, relation_type + 1, " ".join(lammps_bond))
        dat.append(f"{i} {relation_type + 1} {' '.join(lammps_bond)}\n")
    
    return dat

def write_lammps_topology(filename, topo_dat):
    with open(filename, "w") as f:
        for line in topo_dat:
            f.write(line)

In [7]:
dat = get_lammps_topology(tops[0], forcefield, "Bonds")
write_lammps_topology("xlig_bonds_test.dat", dat)
dat[0:5]

['1 1 1 2\n', '2 1 1 31\n', '3 2 1 32\n', '4 2 1 33\n', '5 1 2 3\n']

In [8]:
def generate_neighbors(topology,structure):
    neighbors = {atom: set() for atom in range(1,len(structure)+1)}
    for relation in topology:
        neighbors[relation[0] + 1].add(relation[-1] + 1)
        neighbors[relation[-1] + 1].add(relation[0] + 1)
    return neighbors

In [9]:
def get_special_bonds(neighbors,structure):
    special_bonds = {atom: set() for atom in range(1, len(structure) + 1)}
    for top in ['Bonds', 'Angles', 'Dihedrals', 'Impropers']:
        for atom in special_bonds:
            special_bonds[atom] = special_bonds[atom].union(neighbors[top][atom])
    
    return special_bonds


def get_special_bond_counts(neighbors, structure):
    special_bond_counts = {atom: [] for atom in range(1, len(structure) + 1)}
    for top in ['Bonds', 'Angles', 'Dihedrals']:
        for atom in special_bond_counts:
            if top == "Bonds" or top == "Angles":
                special_bond_counts[atom].append(len(neighbors[top][atom]))
            elif top == "Dihedrals":
                count = 0
                for i in neighbors[top][atom]:
                    if i not in neighbors["Angles"][atom]:
                        count += 1
                special_bond_counts[atom].append(count)
    
    return special_bond_counts

In [16]:
tops[0].as_dict()['sites']

{'@module': 'pymatgen.core.structure',
 '@class': 'Molecule',
 'charge': 0.0,
 'spin_multiplicity': 1,
 'sites': [{'name': 'C',
   'species': [{'element': 'C', 'occu': 1}],
   'xyz': [23.52343, 33.65611, 53.99145],
   'properties': {'ff_map': 'C1'}},
  {'name': 'C',
   'species': [{'element': 'C', 'occu': 1}],
   'xyz': [23.15767, 32.34818, 53.26003],
   'properties': {'ff_map': 'C1'}},
  {'name': 'C',
   'species': [{'element': 'C', 'occu': 1}],
   'xyz': [24.4817, 31.85062, 52.64144],
   'properties': {'ff_map': 'C1'}},
  {'name': 'N',
   'species': [{'element': 'N', 'occu': 1}],
   'xyz': [25.54171, 32.46851, 53.47921],
   'properties': {'ff_map': 'N1'}},
  {'name': 'C',
   'species': [{'element': 'C', 'occu': 1}],
   'xyz': [26.75328, 32.51001, 52.79901],
   'properties': {'ff_map': 'C2'}},
  {'name': 'C',
   'species': [{'element': 'C', 'occu': 1}],
   'xyz': [27.64595, 33.55293, 53.03291],
   'properties': {'ff_map': 'C2'}},
  {'name': 'C',
   'species': [{'element': 'C', 'occu':

In [17]:
topologies = ['Bonds', 'Angles', 'Dihedrals', 'Impropers']
neighbors = {topology: generate_neighbors(tops[0].topologies[topology], xlig) for topology in topologies}
special_bonds = get_special_bonds(neighbors,xlig)
special_bond_counts = get_special_bond_counts(neighbors,xlig)

In [18]:
neighbors['Impropers']

{1: set(),
 2: set(),
 3: set(),
 4: {5},
 5: {4, 6, 30},
 6: {5, 7, 38},
 7: {6, 8, 39},
 8: {7, 9, 29},
 9: {8, 10, 11},
 10: {9},
 11: {9, 12, 28},
 12: {11, 13, 40},
 13: {12, 14, 41},
 14: {13, 15, 27},
 15: {14},
 16: set(),
 17: set(),
 18: set(),
 19: set(),
 20: set(),
 21: set(),
 22: set(),
 23: set(),
 24: set(),
 25: set(),
 26: set(),
 27: {14, 28, 63},
 28: {11, 27, 64},
 29: {8, 30, 65},
 30: {5, 29, 66},
 31: set(),
 32: set(),
 33: set(),
 34: set(),
 35: set(),
 36: set(),
 37: set(),
 38: {6},
 39: {7},
 40: {12},
 41: {13},
 42: set(),
 43: set(),
 44: set(),
 45: set(),
 46: set(),
 47: set(),
 48: set(),
 49: set(),
 50: set(),
 51: set(),
 52: set(),
 53: set(),
 54: set(),
 55: set(),
 56: set(),
 57: set(),
 58: set(),
 59: set(),
 60: set(),
 61: set(),
 62: set(),
 63: {27},
 64: {28},
 65: {29},
 66: {30},
 67: set(),
 68: set()}

In [19]:
coords = []
types = []
charges = []
with open("coords.txt") as f:
    dat = f.readlines()
    
for line in dat:
    raw = line.split()
    x = round(float(raw[3]),1)
    y = round(float(raw[4]),1)
    z = round(float(raw[5]),1)
    pos = [int(raw[0]), x,y,z]
    at = [int(raw[0]), int(raw[2])]
    charge = [int(raw[0]), float(raw[6])]
    charges.append(charge)
    types.append(at)
    coords.append(pos)

bonds = get_lammps_topology(tops[0], forcefield, 'Bonds')
angles = get_lammps_topology(tops[0], forcefield, 'Angles')
dihedrals = get_lammps_topology(tops[0], forcefield, 'Dihedrals')
impropers = get_lammps_topology(tops[0], forcefield, 'Impropers')

In [22]:
with open("xlig.lammps.txt", "w") as f:
    
    f.write("Coords\n\n")
    for pos in coords:
        f.write(f"{pos[0]}    {pos[1]} {pos[2]} {pos[3]}\n")
        
    f.write("\nTypes\n\n")
    for at in types:
        f.write(f"{at[0]}    {at[1]}\n")
        
    f.write("\nCharges\n\n")
    for charge in charges:
        f.write(f"{charge[0]}    {charge[1]}\n")
    
    f.write("\nBonds\n\n")
    for bond in bonds:
        f.write(bond)
    
    f.write("\nAngles\n\n")
    for angle in angles:
        f.write(angle)
    
    f.write("\nDihedrals\n\n")
    for dihed in dihedrals:
        f.write(dihed)
        
    f.write("\nImpropers\n\n")
    for imp in impropers:
        f.write(imp)
    
    f.write("\nSpecial Bond Counts\n\n")
    for atom in special_bond_counts:
        f.write(f"{atom}    {' '.join([str(x) for x in special_bond_counts[atom]])}\n")
        
    f.write("\nSpecial Bonds\n\n")
    for atom in special_bonds:
        f.write(f"{atom}    {' '.join([str(x) for x in special_bonds[atom]])}\n")

In [17]:
len(tops[1].topologies["Bonds"])

53

In [19]:
topologies = ['Bonds', 'Angles', 'Dihedrals', 'Impropers']
neighbors = {topology: generate_neighbors(tops[1].topologies[topology], oleic) for topology in topologies}
special_bonds = get_special_bonds(neighbors, oleic)
special_bond_counts = get_special_bond_counts(neighbors, oleic)

In [22]:
neighbors['Impropers']

{1: {20},
 2: {20},
 3: set(),
 4: set(),
 5: set(),
 6: set(),
 7: set(),
 8: set(),
 9: set(),
 10: set(),
 11: {15},
 12: set(),
 13: set(),
 14: {17},
 15: {11, 17, 45},
 16: set(),
 17: {14, 15, 48},
 18: {20},
 19: set(),
 20: {1, 2, 18},
 21: set(),
 22: set(),
 23: set(),
 24: set(),
 25: set(),
 26: set(),
 27: set(),
 28: set(),
 29: set(),
 30: set(),
 31: set(),
 32: set(),
 33: set(),
 34: set(),
 35: set(),
 36: set(),
 37: set(),
 38: set(),
 39: set(),
 40: set(),
 41: set(),
 42: set(),
 43: set(),
 44: set(),
 45: {15},
 46: set(),
 47: set(),
 48: {17},
 49: set(),
 50: set(),
 51: set(),
 52: set(),
 53: set(),
 54: set()}

In [20]:
coords = []
types = []
charges = []
with open("oleic_coords.txt") as f:
    dat = f.readlines()
    
for line in dat:
    raw = line.split()
    x = round(float(raw[3]),1)
    y = round(float(raw[4]),1)
    z = round(float(raw[5]),1)
    pos = [int(raw[0]), x,y,z]
    at = [int(raw[0]), int(raw[2])]
    charge = [int(raw[0]), float(raw[6])]
    charges.append(charge)
    types.append(at)
    coords.append(pos)

with open("oleic_dat.txt", "w") as f:
    f.write("Coords\n\n")
    for pos in coords:
        f.write(f"{pos[0]}    {pos[1]} {pos[2]} {pos[3]}\n")
    f.write("\nTypes\n\n")
    for at in types:
        f.write(f"{at[0]}    {at[1]}\n")
    f.write("\nCharges\n\n")
    for charge in charges:
        f.write(f"{charge[0]}    {charge[1]}\n")
    f.write("\nSpecial Bond Counts\n\n")
    for atom in special_bond_counts:
        f.write(f"{atom}    {' '.join([str(x) for x in special_bond_counts[atom]])}\n")
    f.write("\nSpecial Bonds\n\n")
    for atom in special_bonds:
        f.write(f"{atom}    {' '.join([str(x) for x in special_bonds[atom]])}\n")

In [21]:
bonds

['1 1 1 2\n',
 '2 1 1 31\n',
 '3 2 1 32\n',
 '4 2 1 33\n',
 '5 1 2 3\n',
 '6 2 2 34\n',
 '7 2 2 35\n',
 '8 3 3 4\n',
 '9 2 3 36\n',
 '10 2 3 37\n',
 '11 4 4 5\n',
 '12 3 4 31\n',
 '13 5 5 6\n',
 '14 5 5 30\n',
 '15 5 6 7\n',
 '16 6 6 38\n',
 '17 5 7 8\n',
 '18 6 7 39\n',
 '19 5 8 9\n',
 '20 5 8 29\n',
 '21 7 9 10\n',
 '22 5 9 11\n',
 '23 5 11 12\n',
 '24 5 11 28\n',
 '25 5 12 13\n',
 '26 6 12 40\n',
 '27 5 13 14\n',
 '28 6 13 41\n',
 '29 8 14 15\n',
 '30 5 14 27\n',
 '31 9 15 16\n',
 '32 1 16 17\n',
 '33 2 16 42\n',
 '34 2 16 43\n',
 '35 1 17 18\n',
 '36 2 17 44\n',
 '37 2 17 45\n',
 '38 1 18 19\n',
 '39 2 18 46\n',
 '40 2 18 47\n',
 '41 1 19 20\n',
 '42 2 19 48\n',
 '43 2 19 49\n',
 '44 1 20 21\n',
 '45 2 20 50\n',
 '46 2 20 51\n',
 '47 1 21 22\n',
 '48 2 21 52\n',
 '49 2 21 53\n',
 '50 1 22 23\n',
 '51 2 22 54\n',
 '52 2 22 55\n',
 '53 1 23 24\n',
 '54 2 23 56\n',
 '55 2 23 57\n',
 '56 1 24 25\n',
 '57 2 24 58\n',
 '58 2 24 59\n',
 '59 9 25 26\n',
 '60 2 25 60\n',
 '61 2 25 61\n',
 '